Tutorial from https://www.datacamp.com/tutorial/knowledge-graph-rag

In [1]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Your raw text
text = """Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.
Michael is also an employee at prismaticAI, where he works as a data scientist. He joined the company two years ago after completing his graduate studies.
prismaticAI is a well-known technology company that specializes in developing cutting-edge software solutions and artificial intelligence applications. The company has a diverse workforce of talented individuals from various backgrounds.
Both Sarah and Michael are highly skilled professionals who contribute significantly to prismaticAI's success. They work closely with their respective teams to develop innovative products and services that meet the evolving needs of the company's clients."""

# Wrap in a Document object
documents = [Document(page_content=text)]

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

# Show result
for i, t in enumerate(texts):
    print(f"\nChunk {i+1}:\n{t.page_content}")



Chunk 1:
Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.

Chunk 2:
Michael is also an employee at prismaticAI, where he works as a data scientist. He joined the company two years ago after completing his graduate studies.

Chunk 3:
prismaticAI is a well-known technology company that specializes in developing cutting-edge software solutions and artificial intelligence applications. The company has a diverse workforce of talented

Chunk 4:
of talented individuals from various backgrounds.

Chunk 5:
Both Sarah and Michael are highly skilled professionals who contribute significantly to prismaticAI's success. They work closely with their respective teams to develop innovative products and

Chunk 6:
products and services that meet the evolving needs of the company's clients.


In [62]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

import os
from dotenv import load_dotenv
load_dotenv()


# Initialize LLM
llm = ChatOpenAI(temperature=0)

# Extract Knowledge Graph
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(texts)

/Users/leandratejedor/miniforge3/envs/agents-py11/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1660: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [56]:
try:
    response = llm.invoke("Hello, are you working properly?")
    print("LLM Response:", response)
    print("Test successful! Your LLM is running correctly.")
except Exception as e:
    print("Error connecting to LLM:", e)
    print("Check your API key and network connection.")

LLM Response: content='Hello! Yes, I am functioning properly. How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 14, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BMkHf9pV37IhPnp53mgu9Aqa2yRmm', 'finish_reason': 'stop', 'logprobs': None} id='run-1b031a01-d933-457f-bfe7-2e05110e9dbf-0' usage_metadata={'input_tokens': 14, 'output_tokens': 17, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Test successful! Your LLM is running correctly.


In [ ]:
from langchain_community.graphs import Neo4jGraph

# Store Knowledge Graph in Neo4j
graph_store = Neo4jGraph(refresh_schema=False)
#graph_store.add_graph_documents(graph_documents)

In [44]:
graph_store.refresh_schema()


In [73]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph_store,
    cypher_llm=llm,
    qa_llm=llm,
    #cypher_prompt=CYPHER_GENERATION_PROMPT,
    verbose=True,
    return_intermediate_steps=True,
    allow_dangerous_requests=True
)

In [78]:
response = chain.invoke({"query": "Where does Michael work?"})
print(response)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Michael"})-[:WORKS_AT]->(o:Organization)
RETURN o;
Full Context:
[{'o': {'id': 'Prismaticai'}}]

> Finished chain.
{'query': 'Where does Michael work?', 'result': "I don't know the answer.", 'intermediate_steps': [{'query': 'MATCH (p:Person {id: "Michael"})-[:WORKS_AT]->(o:Organization)\nRETURN o;'}, {'context': [{'o': {'id': 'Prismaticai'}}]}]}
